In [24]:
from database.strategy import Strategy
from database.sec import SEC
from database.market import Market
from extractor.tiingo_extractor import TiingoExtractor
from transformer.model_transformer import ModelTransformer
from transformer.product_transformer import ProductTransformer
from transformer.predictor_transformer import PredictorTransformer
from preprocessor.model_preprocessor import ModelPreprocessor
from preprocessor.predictor_preprocessor import PredictorPreprocessor
from modeler.modeler import Modeler
from utils.date_utils import DateUtils
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from tqdm import tqdm
import math

In [61]:
## Loading Constants
start = ((datetime.now() - timedelta(days=3)) - timedelta(days=int(365.25*5))).strftime("%Y-%m-%d")
end = (datetime.now() - timedelta(days=3)).strftime("%Y-%m-%d")
year = (datetime.now() - timedelta(days=3)).year
quarter = (((datetime.now() - timedelta(days=3)).month - 1) // 3 + 1)
week = datetime.now().isocalendar()[1] - 2
# Loading Databases
strat = "unity_recommendations"
strat_db = Strategy(strat)
market = Market()
sec = SEC("sec")
market.connect()
tickers = market.retrieve_data("sp500")
market.close()
# Activate tests
reload = True
run_quarterly = True
run_weekly = True
run_weekly_signal = True
#### CAHNGE AVALUES
ticker_range = range(46,47)
model_range = range(46,47)

In [62]:
if reload:
    strat_db.connect()
    strat_db.drop_table("prices")
    for ticker in tqdm(tickers["Symbol"]):
        try:
            if "." in ticker:
                ticker = ticker.replace(".","-")
            status = "tiingo"
            tex = TiingoExtractor(ticker,start,end)
            data = tex.extract()
            data["ticker"] = ticker
            data["download_date"] = end
            strat_db.store_data("prices",data)
        except Exception as e:
            msg = {"status":status,"ticker":ticker,"exception":str(e)}
            print(msg)
    strat_db.close()

 99%|█████████████████████████████████████████████████████████████████████████████████████▍| 502/505 [02:32<00:00,  3.13it/s]

If using all scalar values, you must pass an index
{'status': 'tiingo', 'ticker': 'YUM', 'exception': "'NoneType' object does not support item assignment"}
If using all scalar values, you must pass an index
{'status': 'tiingo', 'ticker': 'ZBRA', 'exception': "'NoneType' object does not support item assignment"}
If using all scalar values, you must pass an index

100%|█████████████████████████████████████████████████████████████████████████████████████▋| 503/505 [02:32<00:00,  3.89it/s]


{'status': 'tiingo', 'ticker': 'ZBH', 'exception': "'NoneType' object does not support item assignment"}
If using all scalar values, you must pass an index
{'status': 'tiingo', 'ticker': 'ZION', 'exception': "'NoneType' object does not support item assignment"}
If using all scalar values, you must pass an index

100%|██████████████████████████████████████████████████████████████████████████████████████| 505/505 [02:32<00:00,  3.31it/s]


{'status': 'tiingo', 'ticker': 'ZTS', 'exception': "'NoneType' object does not support item assignment"}


In [63]:
refined_regression = []
refined_daily_classification = []
refined_quarterly_classification = []
refined_weekly_classification = []
strat_db.connect()
if reload:
    for i in tqdm(ticker_range):
        try:
            ticker = tickers["Symbol"][i]
            if "." in ticker:
                ticker = ticker.replace(".","-")
            prices = strat_db.retrieve_price_data("prices",ticker)
            prices.reset_index(inplace=True)
            relev = prices[["date","adjClose"]]
            for column in ["date","adjClose"]:
                relev.rename(columns={column:"".join(column.split("_")).lower()},inplace=True)
            relev["date"] = [datetime.strptime(x.split("T")[0],"%Y-%m-%d") for x in relev["date"]]
            relev["ticker"] = ticker
            relev.sort_values("date",inplace=True)
            ## daily transformations
            refined_regression.append(relev)
            relev_classification = relev.copy()
            relev_classification["adjclose"] = [1 if x > 0 else 0 for x in relev_classification["adjclose"].diff()]
            refined_daily_classification.append(relev_classification)
            ## weekly transformations
            relev["week"] = [x.week for x in relev["date"]]
            relev["year"] = [x.year for x in relev["date"]]
            relev["quarter"] = [x.quarter for x in relev["date"]]
            relev_weekly_classification = relev.groupby(["year","week"]).mean().reset_index()
            relev_weekly_classification["adjclose"] = [1 if x > 0 else 0 for x in relev_weekly_classification["adjclose"].diff()]
            relev_weekly_classification["ticker"] = ticker
            refined_weekly_classification.append(relev_weekly_classification)
            ## quarterly transformations
            relev_quarterly_classification = relev.groupby(["year","quarter"]).mean().reset_index().drop("week",axis=1)
            relev_quarterly_classification["adjclose"] = [1 if x > 0 else 0 for x in relev_quarterly_classification["adjclose"].diff()]
            relev_quarterly_classification["ticker"] = ticker
            refined_quarterly_classification.append(relev_quarterly_classification)
        except Exception as e:
            print(str(e),ticker)
strat_db.close()
classification_sets = {"date":refined_daily_classification,
                       "quarter":refined_quarterly_classification,
                      "week":refined_weekly_classification}

100%|██████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.75it/s]


In [64]:
if reload:
    for dataset in classification_sets:
        base = pd.concat(classification_sets[dataset])
        if dataset == "date":
            final = base.pivot_table(index=dataset,values="adjclose",columns="ticker").reset_index()
            final = final[final["date"] >= datetime.strptime(start,"%Y-%m-%d")]
        else:
            final = base.pivot_table(index=["year",dataset],values="adjclose",columns="ticker").reset_index()
            final = final[final["year"] >= datetime.strptime(start,"%Y-%m-%d").year]
        name = "dataset_{}_classification".format(dataset)
        for column in final.columns:
            if "_" in column:
                final.drop(column,axis=1,inplace=True,errors="ignore")
        final.fillna(-99999,inplace=True)
        for column in tqdm(final.columns):
            if -99999 == final[column].min():
                final = final.drop(column,axis=1)
        strat_db.drop_table(name)
        strat_db.store_data(name,final)

100%|████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 2895.95it/s]


In [65]:
if reload:
    base = pd.concat(refined_regression)
    final = base.pivot_table(index="date",values="adjclose",columns="ticker").reset_index()
    final = final[final["date"] >= datetime.strptime(start,"%Y-%m-%d")]
    for column in final.columns:
        if "_" in column:
            final.drop(column,axis=1,inplace=True,errors="ignore")
    final.fillna(-99999,inplace=True)
    for column in tqdm(final.columns):
        if -99999 == final[column].min():
            final.drop(column,axis=1,inplace=True)
    strat_db.drop_table("dataset_date_regression")
    strat_db.store_data("dataset_date_regression",final)
    final["quarter"] = [x.quarter for x in final["date"]]
    final["year"] = [x.year for x in final["date"]]
    final["week"] = [x.week for x in final["date"]]
    for timeframe in ["week","quarter"]:
        relev = final.groupby(["year",timeframe]).mean().reset_index()
        relev.reset_index(drop=True,inplace=True)
        name = "dataset_{}_regression".format(timeframe)
        strat_db.drop_table(name)
        strat_db.store_data(name,relev)

100%|████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 1996.34it/s]


In [66]:
sims = []
training_days = 100
gap = 90
quarter_gap = int(gap/90)
training_years = 4
training_days = 100
timeline = DateUtils.create_timeline(start,end)
sims = []
sec.connect()
strat_db.connect()
quarterly_classification_data = strat_db.retrieve_data("dataset_quarter_classification")
quarterly_regression_data = strat_db.retrieve_data("dataset_quarter_regression")
price_regression_data = strat_db.retrieve_data("dataset_date_regression")
if run_quarterly:
    strat_db.drop_table("quarterly_rec")
    for y in range(year,year+1):
        for q in range(quarter - 2, quarter- 1):
            try:
                date_ranges = DateUtils.create_quarterly_training_range_rec(y,q,training_years,gap)
                print(date_ranges)
                training_start,training_end,prediction_start,prediction_end = date_ranges
                dates = pd.to_datetime(date_ranges)
                quarters = [x.quarter for x in dates]
                years = [x.year for x in dates]
                ### switch to all tickers
                for i in model_range:
                    try:
                        ticker = tickers.iloc[i]["Symbol"].replace(".","-")
                        if ticker in quarterly_regression_data.columns:
                            cik = int(tickers.iloc[i]["CIK"].item())
                            price = strat_db.retrieve_price_data("prices",ticker)
                            filing = sec.retrieve_filing_data(cik)
                            mt = ModelTransformer(ticker,training_start,training_end,gap)
                            mr = ModelPreprocessor(ticker)
                            sp = Modeler(ticker)
                            prot = ProductTransformer(ticker,prediction_start,prediction_end)
                            ## regression_model
                            
                            rqpd = mt.quarterly_price_transform(quarterly_regression_data,ticker
                                                                                ,years[0],quarters[0],
                                                                               years[1],quarters[1],quarter_gap)
                            qpd = mr.day_trade_preprocess_regression(rqpd.copy(),ticker)
                            q1c = qpd["X"].columns
                            rpr = sp.quarterly_model("prices",qpd,str(training_years),str(gap))
                            ## classification_model
                            cfd = mt.fundamental_merge(price.copy(),filing.copy(),True,classify=True)
                            cqpd = mt.quarterly_price_transform(quarterly_classification_data,ticker
                                                                                ,years[0],quarters[0],
                                                                               years[1],quarters[1],quarter_gap)
                            qpd = mr.day_trade_preprocess_classify(cqpd.copy(),ticker)
                            q2c = qpd["X"].columns
                            cpr = sp.classify_wta("prices",qpd,str(training_years),str(gap))
                            price_results = pd.DataFrame([cpr,rpr])
                            product_refineds = []
                            product_qpds = []
                            for i in range(len(price_results)):
                                price_result = price_results.iloc[i]
                                if price_result["classification"] == False:
                                    quarterly_price_data = quarterly_regression_data
                                else:
                                    quarterly_price_data = quarterly_classification_data
                                quarterly_price_data = quarterly_price_data[(quarterly_price_data["year"] == years[3]) & 
                                                                           (quarterly_price_data["quarter"] == quarters[3])]
                                quarterly_price_data["y"] = quarterly_price_data[ticker]
                                product_qpd = mr.day_trade_preprocess_regression(quarterly_price_data.copy(),ticker)
                                product_qpds.append(product_qpd)
                            sim = prot.merge_quarterlies_price(price.copy(),product_qpds,price_results)
                            strat_db.store_data("quarterly_rec",sim)
                    except Exception as e:
                        message = {"status":"quarterly modeling","ticker":ticker,"year":str(year),"quarter":str(quarter),"message":str(e)}
                        print(message)
            except Exception as e:
                print(y,q,str(e))
sec.close()
strat_db.close()

['2016-04-01', '2020-03-31', '2020-04-01', '2020-06-30']


In [56]:
model_data = quarterly_regression_data
start_year = years[0]
end_year = years[1]
start_quarter = quarters[0]
end_quarter = quarters[1]
first_index = model_data[(model_data["year"] == start_year) & (model_data["quarter"] == start_quarter)].index.values.tolist()[0]
last_index = model_data[(model_data["year"] == end_year) & (model_data["quarter"] == end_quarter)].index.values.tolist()[-1]
ticker_data = model_data.iloc[first_index:last_index+1]

IndexError: list index out of range

2

In [31]:
sims = []
training_days = 100
gap = 365
quarter_gap = int(gap/90)
training_years = 4
training_days = 100
timeline = DateUtils.create_timeline(start,end)
sims = []
sec.connect()
strat_db.connect()
market.connect()
quarterly_classification_data = market.retrieve_data("dataset_quarter_classification")
quarterly_regression_data = market.retrieve_data("dataset_quarter_regression")
price_regression_data = market.retrieve_data("dataset_date_regression")
strat_db.drop_table("fundamental_quarterly_sim")
for y in range(year,year+1):
    for q in range(quarter - 2, quarter + 1):
        try:
            ## Setting Up
            date_ranges = DateUtils.create_quarterly_training_range(timeline,y,q,training_years,gap)
            training_start,training_end,prediction_start,prediction_end = date_ranges
            dates = pd.to_datetime(date_ranges)
            quarters = [x.quarter for x in dates]
            years = [x.year for x in dates]
            ### switch to all tickers
            for i in model_range:
                try:
                    ticker = tickers.iloc[i]["Symbol"].replace(".","-")
                    if ticker in quarterly_regression_data.columns:
                        cik = int(tickers.iloc[i]["CIK"].item())
                        price = market.retrieve_price_data("prices",ticker)
                        filing = sec.retrieve_filing_data(cik)
                        mt = ModelTransformer(ticker,training_start,training_end,gap)
                        mr = ModelPreprocessor(ticker)
                        sp = Modeler(ticker)
                        prot = ProductTransformer(ticker,prediction_start,prediction_end)
                        ## regression_model
                        rfd = mt.fundamental_merge(price.copy(),filing.copy(),True,classify=False)
                        refined = mr.fundamental_preprocess(rfd.copy())
                        rfr = sp.quarterly_model("prices",refined,str(training_years),str(gap))
                        ## classification_model
                        cfd = mt.fundamental_merge(price.copy(),filing.copy(),True,classify=True)
                        refined = mr.fundamental_preprocess(cfd.copy())
                        cfr = sp.classify_wta("prices",refined,str(training_years),str(gap))
                        fundamental_results = pd.DataFrame([cfr,rfr])
                        product_refineds = []
                        product_qpds = []
                        for i in range(len(fundamental_results)):
                            fundamental_result = fundamental_results.iloc[i]
                            if fundamental_result["classification"] == False:
                                fundamental_data = rfd
                            else:
                                fundamental_data = cfd
                            product_refined = mr.fundamental_preprocess(fundamental_data.copy())
                            product_refineds.append(product_refined)
                        sim = prot.merge_quarterlies_fundamental(price.copy(),product_refineds,fundamental_results)
                        strat_db.store_data("fundamental_quarterly_sim",sim)
                except Exception as e:
                    message = {"status":"quarterly modeling","ticker":ticker,"year":str(year),"quarter":str(quarter),"message":str(e)}
                    print(message)
        except Exception as e:
            print(year,quarter,str(e))
sec.close()
market.close()
strat_db.close()

2020 4 single positional indexer is out-of-bounds
2020 4 single positional indexer is out-of-bounds
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a no

unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unit

unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unit

unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unit

unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unity_recommendations fundamental_quarterly_sim documents must be a non-empty list
unit

In [32]:
sims = []
test = True
training_days = 100
gap = 5
week_gap = int(gap/5)
training_years = 4
training_days = 100
daily_gap = 2
timeline = DateUtils.create_timeline(start,end)
sims = []
sec.connect()
strat_db.connect()
classification = strat_db.retrieve_data("dataset_week_classification")
regression = strat_db.retrieve_data("dataset_week_regression")
strat_db.drop_table("weekly_rec")
if run_weekly:
    try:
        ## Setting Up
        date_ranges = DateUtils.create_weekly_training_range(timeline,year,week,training_years)
        training_start,training_end,prediction_start,prediction_end = date_ranges
        dates = pd.to_datetime(date_ranges)
        weeks = [x.week for x in dates]
        years = [x.year for x in dates]
        for i in model_range:
            try:
                ticker = tickers.iloc[i]["Symbol"].replace(".","-")
                if ticker in regression.columns:
                    price = strat_db.retrieve_price_data("prices",ticker)
                    mt = ModelTransformer(ticker,training_start,training_end,week_gap)
                    mr = ModelPreprocessor(ticker)
                    sp = Modeler(ticker)
                    prot = ProductTransformer(ticker,prediction_start,prediction_end)
                    ## regression_model
                    rqpd = mt.weekly_price_transform(regression,ticker
                                                                        ,years[0],weeks[0],
                                                                       years[1],weeks[1],week_gap)
                    qpd = mr.day_trade_preprocess_regression(rqpd.copy(),ticker)
                    q1c = qpd["X"].columns
                    rpr = sp.quarterly_model("prices",qpd,str(training_years),str(week_gap))
                    ## classification_model
                    cqpd = mt.weekly_price_transform(classification,ticker
                                                                        ,years[0],weeks[0],
                                                                       years[1],weeks[1],week_gap)
                    qpd = mr.day_trade_preprocess_classify(cqpd.copy(),ticker)
                    q2c = qpd["X"].columns
                    cpr = sp.classify_wta("prices",qpd,str(training_years),str(week_gap))
                    price_results = pd.DataFrame([cpr,rpr])
                    product_qpds = []
                    for i in range(len(price_results)):
                        price_result = price_results.iloc[i]
                        if price_result["classification"] == False:
                            weekly_price_data = regression
                        else:
                            weekly_price_data = classification
                        weekly_price_data = weekly_price_data[(weekly_price_data["year"] == years[3]) & 
                                                                   (weekly_price_data["week"] == weeks[3])]
                        weekly_price_data["y"] = weekly_price_data[ticker]
                        product_qpd = mr.day_trade_preprocess_regression(weekly_price_data.copy(),ticker)
                        product_qpds.append(product_qpd)
                    sim = prot.merge_weeklies(price.copy(),product_qpds,price_results)
                    if len(sim) > 0:
                        strat_db.store_data("weekly_rec",sim)
            except Exception as e:
                message = {"status":"weekly modeling","ticker":ticker,"year":str(year),"week":str(week),"message":str(e)}
                print(message)
    except Exception as e:
        print(year,week,str(e))
sec.close()
strat_db.close()

In [33]:
sim

,date,ticker,adjclose,wmtd,weekly_price_classification_prediction,weekly_price_classification_score,weekly_price_regression_prediction,weekly_price_regression_score
1004,2020-12-01 00:00:00+00:00,XYL,97.07,97.07,0.0,0.673077,100.698669,0.939921
1005,2020-12-02 00:00:00+00:00,XYL,96.14,97.07,0.0,0.673077,100.698669,0.939921
1006,2020-12-03 00:00:00+00:00,XYL,96.75,97.07,0.0,0.673077,100.698669,0.939921
1007,2020-12-04 00:00:00+00:00,XYL,98.41,98.41,0.0,0.673077,100.698669,0.939921


In [34]:
strat_db.connect()
weekly_sim = strat_db.retrieve_data("weekly_rec")
price_quarterly_sim = strat_db.retrieve_data("quarterly_rec")
fundamental_quarterly_sim = strat_db.retrieve_data("fundamental_quarterly_rec")
strat_db.close()
quarterly_sim = fundamental_quarterly_sim.drop(["_id","adjclose"],axis=1) \
                    .merge(price_quarterly_sim.drop("_id",axis=1),on=["date","ticker"],how="left")

KeyError: "['_id'] not found in axis"

In [ ]:
weekly_sim["passed"] = weekly_sim["wmtd"] > weekly_sim["weekly_price_regression_prediction"]
quarterly_sim_columns = quarterly_sim.columns
weekly_sim_columns = weekly_sim.columns
qsc = [x for x in quarterly_sim_columns if "score" in x]
wsc = [x for x in weekly_sim_columns if "score" in x]
qcc = [x for x in quarterly_sim_columns if "classification_prediction" in x]
wcc = [x for x in weekly_sim_columns if "classification_prediction" in x]
qrc = [x for x in quarterly_sim_columns if "regression_prediction" in x]
wrc = [x for x in weekly_sim_columns if "regression_prediction" in x]

In [ ]:
availability = []
sp5 = tickers
end_datetime = datetime.strptime(end,"%Y-%m-%d")
for i in model_range:
    ticker = list(sp5["Symbol"])[i]
    ticker = ticker.replace(".","-")
    td = weekly_sim[(weekly_sim["ticker"]==ticker) & (weekly_sim["date"]<=end_datetime)]
    try:
        availability.append({"ticker":ticker
                             ,"end_date":end_datetime <= td.iloc[len(td)-1]["date"]
                             ,"days":(td.iloc[len(td)-1]["date"]- td.iloc[0]["date"]).days
                            ,"records":len(td[td["date"] <= end_datetime])
                            ,"pct":math.ceil(len(td[td["date"] <= end_datetime])/(end_datetime - td.iloc[0]["date"]).days * 100)})
    except Exception as e:
        print(ticker,str(e))
        continue
stuff = pd.DataFrame(availability)
sp5_tickers = list(stuff[(stuff["pct"] >= stuff["pct"].mode().item() - 1) | (stuff["pct"] <= stuff["pct"].mode().item() + 1)]["ticker"].values)

In [ ]:
sims = []
test = True
gap = 5
week_gap = int(gap/5)
training_years = 1
training_days = 100
timeline = DateUtils.create_timeline(start,end)
sims = []
sec.connect()
strat_db.connect()
strat_db.drop_table("rec")
if run_weekly_signal:
    try:
        ## Setting Up
        date_ranges = DateUtils.create_weekly_training_range(timeline,year,week,training_years)
        training_start,training_end,prediction_start,prediction_end = date_ranges
        for i in range(len(sp5_tickers)):
            try:
                ticker = sp5_tickers[i].replace(".","-")
                quarter_sim = quarterly_sim[(quarterly_sim["ticker"] == ticker) & (quarterly_sim["date"] <= prediction_end) & (quarterly_sim["date"] >= training_start)]
                week_sim = weekly_sim[(weekly_sim["ticker"] == ticker) & (weekly_sim["date"] <= prediction_end) & (weekly_sim["date"] >= training_start)]
                for col in wrc:
                    week_sim["{}_delta".format(col)] = (week_sim[col] - week_sim["adjclose"])/week_sim["adjclose"]
                for col in qrc:
                    quarter_sim["{}_delta".format(col)] = (quarter_sim[col] - quarter_sim["adjclose"])/quarter_sim["adjclose"]
                sim = week_sim.drop("_id",axis=1).merge(quarter_sim.drop("adjclose",axis=1),on=["ticker","date"],how="left").dropna()
                training_set = sim[sim["date"] <= training_end]
                prediction_set = sim[sim["date"] >= prediction_start].iloc[:5]
                try:
                    mr = ModelPreprocessor(ticker)
                    sp = Modeler(ticker)
                    mt = ModelTransformer(ticker,start,end,week_gap)
                    ## classification_model
                    ts = mt.trade_signal_transform_classify(training_set.copy())
                    qpd = mr.trade_signal_preprocess_classify(ts.copy())
                    price_result = sp.classify_wta("prices",qpd,str(training_years),str(week_gap))
                except Exception as e:
                    message = {"status":"modeling","ticker":ticker,"message":str(e)}
                    print(message)
                try:
                    weekly_price_data = prediction_set.copy()
                    weekly_price_data["y"] = weekly_price_data["adjclose"]
                    product_qpd = mr.trade_signal_preprocess_classify(weekly_price_data.copy())
                    classification_predictions = price_result["model"].predict(product_qpd["X"])
                    prediction_set["trade_signal_classification_prediction"] = classification_predictions
                    prediction_set["trade_signal_classification_score"] = price_result["score"]
                    product = prediction_set[["date","ticker"
                                              ,"adjclose","weekly_price_regression_prediction"
                                              ,"weekly_price_regression_prediction_delta"
                                              ,"weekly_price_regression_score","trade_signal_classification_prediction"
                                              ,"trade_signal_classification_score"]]
                    if product.index.size > 0:
                        strat_db.store_data("rec",product)
                except Exception as e:
                    message = {"status":"packaging","ticker":ticker,"message":str(e)}
                    print(message)
            except Exception as e:
                message = {"status":"weekly modeling","ticker":ticker,"message":str(e)}
                print(message)
    except Exception as e:
        print(year,week,str(e))
sec.close()
strat_db.close()

In [ ]:
# run quarterly
# store quarterly
# run weekly
# run weekly signal
# store weekly
# store predictions
